# Credit model with Azure Service and scikit-learn

### Contents

 1. Setup
 1. Test the sample model locally
 1. Register the model to your Azure workspace
 1. Deploy the model as Webservice
 1. Test the deployed model

**Note:** Sample model was created using scikit-learn version: `0.20.2`. Model predicts if there is a risk for credit.

## 1. Setup

### Install required packages

In [46]:
!pip install --upgrade scikit-learn==0.20.2 | tail -n 1
!pip install --upgrade azureml-core  | tail -n 1

mlxtend 0.17.2 has requirement scikit-learn>=0.20.3, but you'll have scikit-learn 0.20.2 which is incompatible.
imbalanced-learn 0.6.2 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.20.2 which is incompatible.
ibm-wos-utils 1.2.4 has requirement tqdm==4.32.1, but you'll have tqdm 4.51.0 which is incompatible.
ibm-wos-utils-rsalehin 1.0.0 has requirement tqdm==4.32.1, but you'll have tqdm 4.51.0 which is incompatible.
autoai-libs 1.10.10 has requirement scikit-learn==0.20.3, but you'll have scikit-learn 0.20.2 which is incompatible.
assistant-dialog-skill-analysis 1.1.0 has requirement scikit-learn>=0.21, but you'll have scikit-learn 0.20.2 which is incompatible.


### Action: Restart the kernel

### Get sample model

In [21]:
!rm -rf german_credit_risk_20.joblib
!wget -O german_credit_risk_20.joblib https://github.com/IBM/watson-openscale-samples/blob/main/assets/models/credit_risk/german_credit_risk_20.joblib?raw=true

--2020-11-03 16:26:13--  https://github.com/IBM/watson-openscale-samples/blob/main/assets/models/credit_risk/german_credit_risk.joblib?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/IBM/watson-openscale-samples/raw/main/assets/models/credit_risk/german_credit_risk.joblib [following]
--2020-11-03 16:26:13--  https://github.com/IBM/watson-openscale-samples/raw/main/assets/models/credit_risk/german_credit_risk.joblib
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/models/credit_risk/german_credit_risk.joblib [following]
--2020-11-03 16:26:14--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/models/credit_risk/german_credit_risk.joblib
Resolving raw.githubusercontent.com (

### Load model from local file

In [1]:
import os
from sklearn.externals import joblib

In [2]:
model_name = "german_credit_risk"
model_path = "german_credit_risk_20.joblib" #"german_credit_risk.joblib"

clf = joblib.load(os.path.join(os.getcwd(), model_path))

## 2. Test the sample model locally

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

### Load test data

In [4]:
!rm -rf credit_risk_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/credit_risk/credit_risk_training.csv

--2020-11-03 16:45:30--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/assets/data/credit_risk/credit_risk_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694222 (678K) [text/plain]
Saving to: ‘credit_risk_training.csv’

credit_risk_trainin 100%[===================>] 677.95K  --.-KB/s    in 0.1s    

2020-11-03 16:45:31 (5.41 MB/s) - ‘credit_risk_training.csv’ saved [694222/694222]



In [5]:
data_df = pd.read_csv("credit_risk_training.csv",dtype={'LoanDuration': int, 'LoanAmount': int, 'InstallmentPercent': int, 'CurrentResidenceDuration': int, 
                              'Age': int, 'ExistingCreditsCount': int, 'Dependents': int})
test_data = data_df
test_data = test_data.drop('Risk', axis=1)

### Predict on test data

In [7]:
predictions = clf['postprocessing'](clf['model'].predict(test_data))

print("Model accuracy: {:.3f}".format(accuracy_score(data_df.Risk, predictions)))

Model accuracy: 0.778


## 3. Register the model to your Azure workspace

In [8]:
import os
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.authentication import InteractiveLoginAuthentication

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.17.0 (/Applications/anaconda3/lib/python3.7/site-packages), Requirement.parse('azureml-core==1.0.74.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.17.0 (/Applications/anaconda3/lib/python3.7/site-packages), Requirement.parse('azureml-core==1.0.74.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.17.0 (/Applications/anaconda3/lib/python3.7/site-packages), Requirement.parse('azureml-core==1.0.74.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.run:StepRun._from_dto with excepti

Please provide your Azure ML Service credentials.

In [9]:
az_ml_service_credentials = {'tenant_id': 'fcf67057-50c9-4ad4-98f3-ffca64add9e9',
                             'subscription_id': '744bca72-2299-451c-b682-ed6fb75fb671', 
                             'resource_group': 'ai-ops-squad',
                             'workspace_name': 'wos-ws'}

In [10]:
interactive_auth = InteractiveLoginAuthentication(tenant_id=az_ml_service_credentials['tenant_id'])

ws = Workspace(subscription_id=az_ml_service_credentials['subscription_id'], 
               resource_group=az_ml_service_credentials['resource_group'], 
               workspace_name=az_ml_service_credentials['workspace_name'], 
               auth=interactive_auth)

In [11]:
ws.write_config()

In [12]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: wos-ws
Azure region: southcentralus
Subscription id: 744bca72-2299-451c-b682-ed6fb75fb671
Resource group: ai-ops-squad


In [14]:
from azureml.core.webservice import AciWebservice

credit_risk_deployment_name = 'credit-risk-prediction'
credit_risk_scoring_endpoint = None

webservices = AciWebservice.list(ws)
for service in webservices:
    print('scoring endpoint', service.name)

scoring endpoint credit-risk-rsalehin
scoring endpoint credit-risk-rsalehin-wrapper1


In [16]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration


model = Model.register(workspace=ws,
                       model_name=model_name,                # Name of the registered model in your workspace.
                       model_path=model_path,                # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version='0.20.3',             # Version of scikit-learn used to create the model.
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='credit risk sample scikit model from Watson Studio',
                       tags={'area': 'german_credit_risk', 'type': 'classification'})

print('Name:', model.name)
print('Version:', model.version)

Registering model german_credit_risk
Name: german_credit_risk
Version: 11


## 4. Deploy the model as Webservice

Model deploymen consist of following steps:

 - Create scoring script
 - Create environment file
 - Create configuration file
 - Deploy model in ACI

### Create scoring script

Scoring script must include two required functions:

 - The `init()` function, which typically loads the model into a global object. This function is run only once.
 - The `run(input_data)` function uses the model to predict a value based on the input data.

In [17]:
%%writefile score_azure.py
import os
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('german_credit_risk')
    #model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'german_credit_risk.joblib')
    model = joblib.load(model_path)

def run(input_data):
    try:
        if type(input_data) is str:
            dict_data = json.loads(input_data)
        else:
            dict_data = input_data
            
        data = pd.DataFrame.from_dict(dict_data['input'])   
        predictions = model['postprocessing'](model['model'].predict(data))
        scores = model['model'].predict_proba(data).tolist()
        records = []
        
        for pred, prob in zip(predictions, scores):
            records.append({"Scored Labels": pred, "Scored Probabilities": prob})
 
        result = {'output': records}
        
        return json.dumps(result)
    except Exception as e:
        result = str(e)
        # return error message back to the client
        return json.dumps({"error": result})

Overwriting score_azure.py


### Create environment file

Create an environment file that specifies all of the script's package dependencies.

In [18]:
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core import Environment

environment = Environment('my-sklearn-environment')

environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'joblib',
    'numpy',
    'scikit-learn==0.20.3'
])


### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container.

In [55]:
from azureml.core.webservice import AciWebservice


aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=0.5
                                                ,tags={"data": "german_credit_risk",  "method" : "sklearn"}, 
                                               description='Predict Credit Risk with sklearn from Watson Studio')

### Deploy in ACI

Configure the image and deploy it with following steps:

 - Build an image
 - Register that image under the workspace
 - Send the image to the ACI container
 - Start up a container in ACI using the image
 - Get the web service HTTP endpoint

**Note:** Estimated time to complete: **about 7-8 minutes**.

In [20]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.image import ContainerImage


service_name = 'credit-risk-rsalehin'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script='score_azure.py', environment=environment)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 533 ms, sys: 94.9 ms, total: 628 ms
Wall time: 6min 54s


In [57]:
from azureml.core.webservice import AciWebservice

credit_risk_deployment_name = service_name
credit_risk_scoring_endpoint = None

webservices = AciWebservice.list(ws)
for service in webservices:
    if service.name == credit_risk_deployment_name:
        credit_risk_scoring_endpoint = service.scoring_uri
        
print('scoring endpoint', credit_risk_scoring_endpoint)

scoring endpoint http://4931176f-9943-46a2-857f-dd355d52914a.southcentralus.azurecontainer.io/score


## 5. Test deployed model

### Prepare scoring payload

In [43]:
scoring_data = {"input":[
                {
                "CheckingStatus": "0_to_200", "LoanDuration": 31, "CreditHistory": "credits_paid_to_date", "LoanPurpose": "other",
                "LoanAmount": 1889, "ExistingSavings": "100_to_500", "EmploymentDuration": "less_1", "InstallmentPercent": 3, "Sex": "female",
                "OthersOnLoan": "none", "CurrentResidenceDuration": 3, "OwnsProperty": "savings_insurance", "Age": 32, "InstallmentPlans": "none",
                "Housing": "own", "ExistingCreditsCount": 1, "Job": "skilled", "Dependents": 1, "Telephone": "none", "ForeignWorker": "yes",
                },
                {
                "CheckingStatus": "no_checking", "LoanDuration": 13, "CreditHistory": "credits_paid_to_date", "LoanPurpose": "car_new",
                "LoanAmount": 1389, "ExistingSavings": "100_to_500", "EmploymentDuration": "1_to_4", "InstallmentPercent": 2, "Sex": "male",
                "OthersOnLoan": "none", "CurrentResidenceDuration": 3, "OwnsProperty": "savings_insurance", "Age": 25, "InstallmentPlans": "none",
                "Housing": "own", "ExistingCreditsCount": 2, "Job": "skilled", "Dependents": 2, "Telephone": "none", "ForeignWorker": "yes",
                }]}

### Send scoring request using an HTTP request

In [51]:
import requests
headers = {"Content-Type": "application/json"}
resp = requests.post(credit_risk_scoring_endpoint, json=scoring_data, headers=headers)
resp.json()

'{"output": [{"Scored Labels": "No Risk", "Scored Probabilities": [0.8922524675865824, 0.10774753241341757]}, {"Scored Labels": "No Risk", "Scored Probabilities": [0.8335192848546905, 0.1664807151453095]}]}'

### Next we will integrate this deployment with OpenScale an use wrapper for payload logging